# Mavlink ViewSheen Gimbal Component
> Mavlink ViewSheen Camera Component for sending commands to a viewsheen gimbal on a companion computer or GCS
> The server is normally mounted on the companion computer and the client is on the ground station PC.

In [ ]:
#| default_exp mavlink.vs_gimbal

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

### Gimbal Component
[https://mavlink.io/en/services/gimbal.html](https://mavlink.io/en/services/gimbal.html)  
[https://mavlink.io/en/services/gimbal_v2.html](https://mavlink.io/en/services/gimbal_v2.html)
> Concepts
 - Gimbal Manager and Gimbal Device
 - To accommodate gimbals with varying capabilities, and various hardware setups, "a gimbal" is conceptually split into two parts:

*Gimbal Device:* the actual gimbal device, hardware and software.
*Gimbal Manager:* software to deconflict gimbal messages and commands from different sources, and to abstract the capabilities of the Gimbal Device from gimbal users.
The Gimbal Manager and Gimbal Device expose respective message sets that can be used for: gimbal manager/device discovery, querying capabilities, publishing status, and various types of orientation/attitude control.

The key concept to understand is that a Gimbal Manager has a 1:1 relationship with a particular Gimbal Device, and is the only party on the M

In [ ]:
#| export
import time, os, sys

from UAV.logging import logging
from UAV.mavlink.mavcom import MAVCom
from UAV.mavlink.component import Component, mavutil
# from viewsheen_sdk.gimbal_cntrl import pan_tilt, snapshot,  zoom, VS_IP_ADDRESS, VS_PORT, KeyReleaseThread
from UAV.camera_sdks.viewsheen.gimbal_cntrl import pan_tilt, snapshot,  zoom, VS_IP_ADDRESS, VS_PORT, KeyReleaseThread

import socket

# from UAV.imports import *   # TODO why is this relative import on nbdev_export?


In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
# from pymavlink.dialects.v20 import ardupilotmega as mav
from pymavlink.dialects.v20.ardupilotmega import MAVLink

NAN = float("nan")
GIMBAL_DEVICE_SET_ATTITUDE = 284  # https://mavlink.io/en/messages/common
GIMBAL_MANAGER_SET_MANUAL_CONTROL = 288  # https://mavlink.io/en/messages/common.html#GIMBAL_MANAGER_SET_MANUAL_CONTROL
MAV_CMD_SET_CAMERA_ZOOM = 531  # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
MAV_CMD_IMAGE_START_CAPTURE = 2000  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
MAV_CMD_IMAGE_STOP_CAPTURE = 2001  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
class GimbalClient(Component):
    """Create a Viewsheen mavlink gimbal client component for send commands to a gimbal on a companion computer or GCS """

    def __init__(self,
                 source_component,  # used for component indication
                 mav_type,  # used for heartbeat MAV_TYPE indication
                 debug):  # logging level
        
        super().__init__( source_component=source_component, mav_type=mav_type, debug=debug)
        # self.gimbal_target_component = None
        # self.camera_target_component = None
        
    def send_message(self, msg):
        """Send a message to the gimbal"""
        self.master.mav.send(msg)
        self.log.debug(f"Sent {msg}")
        
    # def set_target(self, target_system, gimbal_comp = None,  camera_comp = None):
    #     """Set the target system and component for the gimbal / camera"""
    #     self.target_system = target_system
    #     self.gimbal_target_component = gimbal_comp
    #     self.camera_target_component = camera_comp
    

    def set_attitude(self, pitch, yaw, pitchspeed, yawspeed):
        """Set the attitude of the gimbal"""
        # https://mavlink.io/en/messages/common.html#GIMBAL_DEVICE_SET_ATTITUDE
        # https://mavlink.io/en/messages/common.html#GIMBAL_DEVICE_FLAGS
        flags = 0

        q = [1, 0, pitch, yaw]
        angular_velocity_x, angular_velocity_y, angular_velocity_z = 0, pitchspeed, yawspeed

        
        # self.mav.gimbal_manager_set_attitude_send(
        #     self.target_system, self.target_component,
        #     flags,
        #     0, # gimbal_device_id , 0=all gimbal components
        #     q,
        #     angular_velocity_x, angular_velocity_y, angular_velocity_z,
        # )

        self.mav.gimbal_device_set_attitude_send(
            self.target_system, self.target_component,
            flags,
            q,
            angular_velocity_x, angular_velocity_y, angular_velocity_z,
        )   
    
    def set_zoom(self, value):
        """ Set the camera zoom"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
        t = self.send_command(self.target_system, self.target_component,
        MAV_CMD_SET_CAMERA_ZOOM,
        [0,
         value, 0,0,0,0,0])
    
    def start_capture(self):
        """Start image capture sequence."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
        t = self.send_command(self.target_system, self.target_component,
        MAV_CMD_IMAGE_START_CAPTURE,
        [0,
         0, # interval
         1, # number of  images to capture
         0, # Sequence number starting from 1. This is only valid for single-capture (param3 == 1), otherwise set to 0.  Increment the capture ID for each capture command to prevent double captures when a command is re-transmitted.
         NAN, # Reserved
         NAN, # Reserved
         NAN]) # Reserved
    
    def stop_capture(self):
        """Stop image capture sequence"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
        t = self.send_command(self.target_system, self.target_component,
        MAV_CMD_IMAGE_STOP_CAPTURE,
        [0, NAN, NAN, NAN, NAN, NAN, NAN])


In [ ]:
show_doc(GimbalClient)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/vs_gimbal.py#L29){target="_blank" style="float:right; font-size:smaller"}

### GimbalClient

>      GimbalClient (mav_connection, source_component, mav_type, debug)

Create a Viewsheen mavlink gimbal client component for send commands to a gimbal on a companion computer or GCS

|    | **Details** |
| -- | ----------- |
| mav_connection | MavLinkBase connection |
| source_component | used for component indication |
| mav_type | used for heartbeat MAV_TYPE indication |
| debug | logging level |

In [ ]:
#| export
class GimbalServer(Component):
    """Create a Viewsheen mavlink Camera Server Component for receiving commands from a gimbal on a companion computer or GCS"""

    def __init__(self,
                 source_component,  # used for component indication
                 mav_type,  # used for heartbeat MAV_TYPE indication
                 debug):  # logging level
        
        super().__init__( source_component=source_component, mav_type=mav_type, debug=debug)
        
        self.set_message_callback(self.on_message)
        self.connect()
     
     
    def connect(self):
        """Connect to the viewsheen_sdk gimbal"""
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.connect((VS_IP_ADDRESS, VS_PORT))
        self.log.debug(f"Connected to gimbal")
        return True
    
    def on_message(self, msg):
        """Callback for a command received from the gimbal"""
        # https://mavlink.io/en/messages/common.html#GIMBAL_DEVICE_SET_ATTITUDE
        # print(f" {msg = }")
        # print(f" {msg.get_type() = }")
        # return False
        if msg.get_type() == "GIMBAL_DEVICE_SET_ATTITUDE" or msg.get_type() == "GIMBAL_MANAGER_SET_ATTITUDE":
            self.set_attitude(msg)
            return False
        elif msg.get_type() == "COMMAND_LONG":
            # print(f"Command  {msg.command = } ")
            if msg.command == MAV_CMD_SET_CAMERA_ZOOM:
                # self.log.info(f"***** Zoom {msg}")
                # print(f"Zoom {msg.param2 = }")
                self.set_zoom(msg)
                return True
            elif msg.command == MAV_CMD_IMAGE_START_CAPTURE:
                self.start_capture()
                return True
            elif msg.command == MAV_CMD_IMAGE_STOP_CAPTURE:
                self.stop_capture()
                return True
            
        else:
            self.log.debug(f"Unknown command {msg.get_type()} received from {msg.get_srcSystem()}/{msg.get_srcComponent()}")
            return False
        
    def set_zoom(self, msg):
        """ Set the viewsheen camera zoom """
        # print(msg.get_type())
        # print(f"Zoom {msg.param2 = }")
        data = zoom(int(msg.param2))
        self.sock.sendall(data)
        
        
    def start_capture(self):
        """Start image capture sequence."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
        data = snapshot(1, 0)
        self.sock.sendall(data)

        
    def set_attitude(self, msg):
        """Set the attitude of the gimbal"""
        # https://mavlink.io/en/messages/common.html#GIMBAL_DEVICE_SET_ATTITUDE
   
        pitch, yaw = msg.q[2], msg.q[3]
        pitchspeed, yawspeed = msg.angular_velocity_y, msg.angular_velocity_z
        pan = int(yawspeed * 100)
        tilt = int(pitchspeed * 100)
        data = pan_tilt(pan, tilt)
        self.sock.sendall(data)
        
    def close(self):
        """Close the connection to the gimbal"""
        super().close()
        self.sock.close()
        self.log.debug(f"Closed connection to gimbal")
        return True


In [ ]:
show_doc(GimbalServer)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/vs_gimbal.py#L108){target="_blank" style="float:right; font-size:smaller"}

### GimbalServer

>      GimbalServer (mav_connection, source_component, mav_type, debug)

Create a Viewsheen mavlink Camera Server Component for receiving commands from a gimbal on a companion computer or GCS

|    | **Details** |
| -- | ----------- |
| mav_connection | MavLinkBase connection |
| source_component | used for component indication |
| mav_type | used for heartbeat MAV_TYPE indication |
| debug | logging level |

In [ ]:
MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA
# cli = GimbalClient(mav_connection=None, source_component=11, mav_type=MAV_TYPE_GCS, debug=False)
# gim1 = GimbalServer(mav_connection=None, source_component=22, mav_type=MAV_TYPE_CAMERA, debug=False)

con1, con2 = "udpin:localhost:14445", "udpout:localhost:14445"
# con1, con2 = "/dev/ttyACM0", "/dev/ttyUSB0"
with MAVCom(con1, source_system=111, debug=False) as client:
    with MAVCom(con2, source_system=222, debug=False) as server:
        gimbal:GimbalClient = client.add_component(GimbalClient(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(GimbalServer(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        
        gimbal.wait_heartbeat(target_system=222, target_component=22, timeout=0.99)
        time.sleep(0.1)
        gimbal.set_target(222, 22)
        
        NAN = float("nan")
        # client.component[11]._test_command(222, 22, 1)
        # for i in range (1)  :
        #     time.sleep(0.01)
        gimbal.set_attitude( NAN, NAN, 0.0, 0.2)
        time.sleep(0.5)
        gimbal.set_attitude( NAN, NAN, 0.0, -0.2)
        time.sleep(0.5)
        gimbal.start_capture()
        # gimbal.set_zoom(1)
        
        
        # client.component[11].set_attitude(0, 0, 0, 0, 0, 0)



INFO   | uav.MAVCom      | 23.021 |  mavcom.py:368 | Thread-17 (listen) | MAVLink Mav2: True, source_system: 111
INFO   | uav.MAVCom      | 23.123 |  mavcom.py:368 | Thread-18 (listen) | MAVLink Mav2: True, source_system: 222
INFO   | uav.GimbalClien | 23.125 | component.py: 78 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
INFO   | uav.GimbalServe | 23.127 | component.py: 78 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222
INFO   | uav.GimbalServe | 25.135 | component.py:365 | MainThread         | GimbalServer closed
INFO   | uav.MAVCom      | 25.137 |  mavcom.py:413 | MainThread         | MAVCom  closed
INFO   | uav.GimbalClien | 26.135 | component.py:365 | MainThread         | GimbalClient closed
INFO   | uav.MAVCom      | 26.137 |  mavcom.py:413 | MainThread         | MAVCom  closed


In [ ]:
# | hide
assert False, "Stop here"

AssertionError: Stop here

In [ ]:
show_doc(Component)

In [ ]:
#| Hide
# assert False, "Stop here"

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()